## Write functions that takes in old and new systems and computes number of ions to turn off, then identifies the ion index to turn off (needs to be > 5 angstroms from protein)

In [1]:
import pickle
import os
import mdtraj as md
import numpy as np

In [2]:
# Define charged amino acids
positive = ['ARG', 'HIS', 'LYS']
negative = ['ASP', 'GLU']

# Get the residue object and residue name of the mutated residue, given the topology and the resid
def get_name(topology, resid):
    for res in topology.residues():
        if res.id == resid:
            return res, res.name
    raise Exception(f"Could not find resid {resid}!")

# Get the charge of an residue, given its name
def get_charge(res_name):
    if res_name in positive:
        return 1
    elif res_name in negative: 
        return -1
    else:
        return 0

# Determine how many ions needs to be turned off
def get_num_ions(old_charge, new_charge):
    charge_difference = old_charge - new_charge
    if charge_difference == 0:
        return 0
    else:
        return charge_difference*-1

# positive to neutral -- turn off a negative charge
# positive to negative -- turn off 2 negative charges
# positive to positive -- do nothing
# negative to neutral -- turn off a positive charge
# negative to positive -- turn off 2 positive charges
# negative to negative -- do nothing
# neutral to positive -- turn off a positive charge
# neutral to negative -- turn off a negative charge
# neutral to neutral -- do nothing

In [3]:
# Identify indices of ion(s) to turn off 
def get_indices(mutated_res, positions, topology, num_ions, positive_ion_name='NA', negative_ion_name='CL', radius=0.5):
    # Retrieve atom indices of mutated residue for querying neighboring atoms
    query_indices = [atom.index for atom in mutated_res.atoms()]
    
    # Create trajectory
    traj = md.Trajectory(np.array(positions), topology)
    
    # Get ion atom indices
    if num_ions < 0:
        ion_name = negative_ion_name
    elif num_ions > 0:
        ion_name = positive_ion_name
    atoms = list(traj.topology.select(f"resname {ion_name}"))
    
    # Get ion indices within radius of mutated residue
    neighboring_ions = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=atoms)[0])
    
    # Get ion indices outside of radius of mutated residue
    nonneighboring_ions = list(set(atoms) - set(neighboring_ions))
    
    return nonneighboring_ions[:abs(num_ions)] # Return first N ion indices, where N = abs(num_ions)

## Test on K27A

In [4]:
# Read in htf
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/12/56"
htf = pickle.load(open(os.path.join(out_dir, f"56_apo_0.pickle"), "rb" ))

FileNotFoundError: [Errno 2] No such file or directory: '/data/chodera/zhangi/perses_benchmark/neq/12/56/56_apo_0.pickle'

In [5]:
old_topology = htf._topology_proposal.old_topology
new_topology = htf._topology_proposal.new_topology
resid = '27'

In [6]:
# Determine charge of old amino acid
old_res, old_res_name = get_name(old_topology, resid)
old_res_charge = get_charge(old_res_name)

In [7]:
old_res_name

'LYS'

In [8]:
old_res_charge

1

In [9]:
# Determine charge of new amino acid
new_res, new_res_name = get_name(new_topology, resid)
new_res_charge = get_charge(new_res_name)

In [10]:
new_res_name

'ALA'

In [11]:
new_res_charge

0

In [12]:
# Get number and type of ions that need to be turned off
num_ions = get_num_ions(old_res_charge, new_res_charge)

In [13]:
num_ions

-1

In [17]:
# Get indices of atoms in the new system that need to be turn off
get_indices(new_res, np.array(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(new_topology), num_ions)

[18085]

In [18]:
for atom in new_topology.atoms():
    print(atom, atom.residue.name)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)> ACE
<Atom 1 (C) of chain 0 residue 0 (ACE)> ACE
<Atom 2 (O) of chain 0 residue 0 (ACE)> ACE
<Atom 3 (H1) of chain 0 residue 0 (ACE)> ACE
<Atom 4 (H2) of chain 0 residue 0 (ACE)> ACE
<Atom 5 (H3) of chain 0 residue 0 (ACE)> ACE
<Atom 6 (N) of chain 0 residue 1 (VAL)> VAL
<Atom 7 (CA) of chain 0 residue 1 (VAL)> VAL
<Atom 8 (C) of chain 0 residue 1 (VAL)> VAL
<Atom 9 (O) of chain 0 residue 1 (VAL)> VAL
<Atom 10 (CB) of chain 0 residue 1 (VAL)> VAL
<Atom 11 (CG1) of chain 0 residue 1 (VAL)> VAL
<Atom 12 (CG2) of chain 0 residue 1 (VAL)> VAL
<Atom 13 (H) of chain 0 residue 1 (VAL)> VAL
<Atom 14 (HA) of chain 0 residue 1 (VAL)> VAL
<Atom 15 (HB) of chain 0 residue 1 (VAL)> VAL
<Atom 16 (HG11) of chain 0 residue 1 (VAL)> VAL
<Atom 17 (HG12) of chain 0 residue 1 (VAL)> VAL
<Atom 18 (HG13) of chain 0 residue 1 (VAL)> VAL
<Atom 19 (HG21) of chain 0 residue 1 (VAL)> VAL
<Atom 20 (HG22) of chain 0 residue 1 (VAL)> VAL
<Atom 21 (HG23) of chain 0 residue 1 

<Atom 1593 (CA) of chain 0 residue 104 (PHE)> PHE
<Atom 1594 (C) of chain 0 residue 104 (PHE)> PHE
<Atom 1595 (O) of chain 0 residue 104 (PHE)> PHE
<Atom 1596 (CB) of chain 0 residue 104 (PHE)> PHE
<Atom 1597 (CG) of chain 0 residue 104 (PHE)> PHE
<Atom 1598 (CD1) of chain 0 residue 104 (PHE)> PHE
<Atom 1599 (CD2) of chain 0 residue 104 (PHE)> PHE
<Atom 1600 (CE1) of chain 0 residue 104 (PHE)> PHE
<Atom 1601 (CE2) of chain 0 residue 104 (PHE)> PHE
<Atom 1602 (CZ) of chain 0 residue 104 (PHE)> PHE
<Atom 1603 (H) of chain 0 residue 104 (PHE)> PHE
<Atom 1604 (HA) of chain 0 residue 104 (PHE)> PHE
<Atom 1605 (HB3) of chain 0 residue 104 (PHE)> PHE
<Atom 1606 (HB2) of chain 0 residue 104 (PHE)> PHE
<Atom 1607 (HD1) of chain 0 residue 104 (PHE)> PHE
<Atom 1608 (HD2) of chain 0 residue 104 (PHE)> PHE
<Atom 1609 (HE1) of chain 0 residue 104 (PHE)> PHE
<Atom 1610 (HE2) of chain 0 residue 104 (PHE)> PHE
<Atom 1611 (HZ) of chain 0 residue 104 (PHE)> PHE
<Atom 1612 (N) of chain 0 residue 105 (THR)

<Atom 3593 (O) of chain 2 residue 742 (HOH)> HOH
<Atom 3594 (H1) of chain 2 residue 742 (HOH)> HOH
<Atom 3595 (H2) of chain 2 residue 742 (HOH)> HOH
<Atom 3596 (O) of chain 2 residue 743 (HOH)> HOH
<Atom 3597 (H1) of chain 2 residue 743 (HOH)> HOH
<Atom 3598 (H2) of chain 2 residue 743 (HOH)> HOH
<Atom 3599 (O) of chain 2 residue 744 (HOH)> HOH
<Atom 3600 (H1) of chain 2 residue 744 (HOH)> HOH
<Atom 3601 (H2) of chain 2 residue 744 (HOH)> HOH
<Atom 3602 (O) of chain 2 residue 745 (HOH)> HOH
<Atom 3603 (H1) of chain 2 residue 745 (HOH)> HOH
<Atom 3604 (H2) of chain 2 residue 745 (HOH)> HOH
<Atom 3605 (O) of chain 2 residue 746 (HOH)> HOH
<Atom 3606 (H1) of chain 2 residue 746 (HOH)> HOH
<Atom 3607 (H2) of chain 2 residue 746 (HOH)> HOH
<Atom 3608 (O) of chain 2 residue 747 (HOH)> HOH
<Atom 3609 (H1) of chain 2 residue 747 (HOH)> HOH
<Atom 3610 (H2) of chain 2 residue 747 (HOH)> HOH
<Atom 3611 (O) of chain 2 residue 748 (HOH)> HOH
<Atom 3612 (H1) of chain 2 residue 748 (HOH)> HOH
<Atom 3

<Atom 5592 (H1) of chain 2 residue 1408 (HOH)> HOH
<Atom 5593 (H2) of chain 2 residue 1408 (HOH)> HOH
<Atom 5594 (O) of chain 2 residue 1409 (HOH)> HOH
<Atom 5595 (H1) of chain 2 residue 1409 (HOH)> HOH
<Atom 5596 (H2) of chain 2 residue 1409 (HOH)> HOH
<Atom 5597 (O) of chain 2 residue 1410 (HOH)> HOH
<Atom 5598 (H1) of chain 2 residue 1410 (HOH)> HOH
<Atom 5599 (H2) of chain 2 residue 1410 (HOH)> HOH
<Atom 5600 (O) of chain 2 residue 1411 (HOH)> HOH
<Atom 5601 (H1) of chain 2 residue 1411 (HOH)> HOH
<Atom 5602 (H2) of chain 2 residue 1411 (HOH)> HOH
<Atom 5603 (O) of chain 2 residue 1412 (HOH)> HOH
<Atom 5604 (H1) of chain 2 residue 1412 (HOH)> HOH
<Atom 5605 (H2) of chain 2 residue 1412 (HOH)> HOH
<Atom 5606 (O) of chain 2 residue 1413 (HOH)> HOH
<Atom 5607 (H1) of chain 2 residue 1413 (HOH)> HOH
<Atom 5608 (H2) of chain 2 residue 1413 (HOH)> HOH
<Atom 5609 (O) of chain 2 residue 1414 (HOH)> HOH
<Atom 5610 (H1) of chain 2 residue 1414 (HOH)> HOH
<Atom 5611 (H2) of chain 2 residue 14

<Atom 7842 (H1) of chain 2 residue 2158 (HOH)> HOH
<Atom 7843 (H2) of chain 2 residue 2158 (HOH)> HOH
<Atom 7844 (O) of chain 2 residue 2159 (HOH)> HOH
<Atom 7845 (H1) of chain 2 residue 2159 (HOH)> HOH
<Atom 7846 (H2) of chain 2 residue 2159 (HOH)> HOH
<Atom 7847 (O) of chain 2 residue 2160 (HOH)> HOH
<Atom 7848 (H1) of chain 2 residue 2160 (HOH)> HOH
<Atom 7849 (H2) of chain 2 residue 2160 (HOH)> HOH
<Atom 7850 (O) of chain 2 residue 2161 (HOH)> HOH
<Atom 7851 (H1) of chain 2 residue 2161 (HOH)> HOH
<Atom 7852 (H2) of chain 2 residue 2161 (HOH)> HOH
<Atom 7853 (O) of chain 2 residue 2162 (HOH)> HOH
<Atom 7854 (H1) of chain 2 residue 2162 (HOH)> HOH
<Atom 7855 (H2) of chain 2 residue 2162 (HOH)> HOH
<Atom 7856 (O) of chain 2 residue 2163 (HOH)> HOH
<Atom 7857 (H1) of chain 2 residue 2163 (HOH)> HOH
<Atom 7858 (H2) of chain 2 residue 2163 (HOH)> HOH
<Atom 7859 (O) of chain 2 residue 2164 (HOH)> HOH
<Atom 7860 (H1) of chain 2 residue 2164 (HOH)> HOH
<Atom 7861 (H2) of chain 2 residue 21

<Atom 10092 (H1) of chain 2 residue 2908 (HOH)> HOH
<Atom 10093 (H2) of chain 2 residue 2908 (HOH)> HOH
<Atom 10094 (O) of chain 2 residue 2909 (HOH)> HOH
<Atom 10095 (H1) of chain 2 residue 2909 (HOH)> HOH
<Atom 10096 (H2) of chain 2 residue 2909 (HOH)> HOH
<Atom 10097 (O) of chain 2 residue 2910 (HOH)> HOH
<Atom 10098 (H1) of chain 2 residue 2910 (HOH)> HOH
<Atom 10099 (H2) of chain 2 residue 2910 (HOH)> HOH
<Atom 10100 (O) of chain 2 residue 2911 (HOH)> HOH
<Atom 10101 (H1) of chain 2 residue 2911 (HOH)> HOH
<Atom 10102 (H2) of chain 2 residue 2911 (HOH)> HOH
<Atom 10103 (O) of chain 2 residue 2912 (HOH)> HOH
<Atom 10104 (H1) of chain 2 residue 2912 (HOH)> HOH
<Atom 10105 (H2) of chain 2 residue 2912 (HOH)> HOH
<Atom 10106 (O) of chain 2 residue 2913 (HOH)> HOH
<Atom 10107 (H1) of chain 2 residue 2913 (HOH)> HOH
<Atom 10108 (H2) of chain 2 residue 2913 (HOH)> HOH
<Atom 10109 (O) of chain 2 residue 2914 (HOH)> HOH
<Atom 10110 (H1) of chain 2 residue 2914 (HOH)> HOH
<Atom 10111 (H2) o

<Atom 12342 (H1) of chain 2 residue 3658 (HOH)> HOH
<Atom 12343 (H2) of chain 2 residue 3658 (HOH)> HOH
<Atom 12344 (O) of chain 2 residue 3659 (HOH)> HOH
<Atom 12345 (H1) of chain 2 residue 3659 (HOH)> HOH
<Atom 12346 (H2) of chain 2 residue 3659 (HOH)> HOH
<Atom 12347 (O) of chain 2 residue 3660 (HOH)> HOH
<Atom 12348 (H1) of chain 2 residue 3660 (HOH)> HOH
<Atom 12349 (H2) of chain 2 residue 3660 (HOH)> HOH
<Atom 12350 (O) of chain 2 residue 3661 (HOH)> HOH
<Atom 12351 (H1) of chain 2 residue 3661 (HOH)> HOH
<Atom 12352 (H2) of chain 2 residue 3661 (HOH)> HOH
<Atom 12353 (O) of chain 2 residue 3662 (HOH)> HOH
<Atom 12354 (H1) of chain 2 residue 3662 (HOH)> HOH
<Atom 12355 (H2) of chain 2 residue 3662 (HOH)> HOH
<Atom 12356 (O) of chain 2 residue 3663 (HOH)> HOH
<Atom 12357 (H1) of chain 2 residue 3663 (HOH)> HOH
<Atom 12358 (H2) of chain 2 residue 3663 (HOH)> HOH
<Atom 12359 (O) of chain 2 residue 3664 (HOH)> HOH
<Atom 12360 (H1) of chain 2 residue 3664 (HOH)> HOH
<Atom 12361 (H2) o

<Atom 14591 (O) of chain 2 residue 4408 (HOH)> HOH
<Atom 14592 (H1) of chain 2 residue 4408 (HOH)> HOH
<Atom 14593 (H2) of chain 2 residue 4408 (HOH)> HOH
<Atom 14594 (O) of chain 2 residue 4409 (HOH)> HOH
<Atom 14595 (H1) of chain 2 residue 4409 (HOH)> HOH
<Atom 14596 (H2) of chain 2 residue 4409 (HOH)> HOH
<Atom 14597 (O) of chain 2 residue 4410 (HOH)> HOH
<Atom 14598 (H1) of chain 2 residue 4410 (HOH)> HOH
<Atom 14599 (H2) of chain 2 residue 4410 (HOH)> HOH
<Atom 14600 (O) of chain 2 residue 4411 (HOH)> HOH
<Atom 14601 (H1) of chain 2 residue 4411 (HOH)> HOH
<Atom 14602 (H2) of chain 2 residue 4411 (HOH)> HOH
<Atom 14603 (O) of chain 2 residue 4412 (HOH)> HOH
<Atom 14604 (H1) of chain 2 residue 4412 (HOH)> HOH
<Atom 14605 (H2) of chain 2 residue 4412 (HOH)> HOH
<Atom 14606 (O) of chain 2 residue 4413 (HOH)> HOH
<Atom 14607 (H1) of chain 2 residue 4413 (HOH)> HOH
<Atom 14608 (H2) of chain 2 residue 4413 (HOH)> HOH
<Atom 14609 (O) of chain 2 residue 4414 (HOH)> HOH
<Atom 14610 (H1) of

<Atom 16841 (O) of chain 2 residue 5158 (HOH)> HOH
<Atom 16842 (H1) of chain 2 residue 5158 (HOH)> HOH
<Atom 16843 (H2) of chain 2 residue 5158 (HOH)> HOH
<Atom 16844 (O) of chain 2 residue 5159 (HOH)> HOH
<Atom 16845 (H1) of chain 2 residue 5159 (HOH)> HOH
<Atom 16846 (H2) of chain 2 residue 5159 (HOH)> HOH
<Atom 16847 (O) of chain 2 residue 5160 (HOH)> HOH
<Atom 16848 (H1) of chain 2 residue 5160 (HOH)> HOH
<Atom 16849 (H2) of chain 2 residue 5160 (HOH)> HOH
<Atom 16850 (O) of chain 2 residue 5161 (HOH)> HOH
<Atom 16851 (H1) of chain 2 residue 5161 (HOH)> HOH
<Atom 16852 (H2) of chain 2 residue 5161 (HOH)> HOH
<Atom 16853 (O) of chain 2 residue 5162 (HOH)> HOH
<Atom 16854 (H1) of chain 2 residue 5162 (HOH)> HOH
<Atom 16855 (H2) of chain 2 residue 5162 (HOH)> HOH
<Atom 16856 (O) of chain 2 residue 5163 (HOH)> HOH
<Atom 16857 (H1) of chain 2 residue 5163 (HOH)> HOH
<Atom 16858 (H2) of chain 2 residue 5163 (HOH)> HOH
<Atom 16859 (O) of chain 2 residue 5164 (HOH)> HOH
<Atom 16860 (H1) of

## Test on A27K

In [19]:
# Read in htf
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/12/57"
htf = pickle.load(open(os.path.join(out_dir, f"57_apo_0.pickle"), "rb" ))

In [20]:
old_topology = htf._topology_proposal.old_topology
new_topology = htf._topology_proposal.new_topology
resid = '27'

In [21]:
# Determine charge of old amino acid
old_res, old_res_name = get_name(old_topology, resid)
old_res_charge = get_charge(old_res_name)

In [22]:
# Determine charge of new amino acid
new_res, new_res_name = get_name(new_topology, resid)
new_res_charge = get_charge(new_res_name)

In [23]:
# Get number and type of ions that need to be turned off
num_ions = get_num_ions(old_res_charge, new_res_charge)

In [24]:
num_ions

1

In [25]:
# Get indices of atoms in the new system that need to be turn off
get_indices(new_res, np.array(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(new_topology), num_ions)

[18096]

In [26]:
for atom in new_topology.atoms():
    print(atom, atom.residue.name)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)> ACE
<Atom 1 (C) of chain 0 residue 0 (ACE)> ACE
<Atom 2 (O) of chain 0 residue 0 (ACE)> ACE
<Atom 3 (H1) of chain 0 residue 0 (ACE)> ACE
<Atom 4 (H2) of chain 0 residue 0 (ACE)> ACE
<Atom 5 (H3) of chain 0 residue 0 (ACE)> ACE
<Atom 6 (N) of chain 0 residue 1 (VAL)> VAL
<Atom 7 (CA) of chain 0 residue 1 (VAL)> VAL
<Atom 8 (C) of chain 0 residue 1 (VAL)> VAL
<Atom 9 (O) of chain 0 residue 1 (VAL)> VAL
<Atom 10 (CB) of chain 0 residue 1 (VAL)> VAL
<Atom 11 (CG1) of chain 0 residue 1 (VAL)> VAL
<Atom 12 (CG2) of chain 0 residue 1 (VAL)> VAL
<Atom 13 (H) of chain 0 residue 1 (VAL)> VAL
<Atom 14 (HA) of chain 0 residue 1 (VAL)> VAL
<Atom 15 (HB) of chain 0 residue 1 (VAL)> VAL
<Atom 16 (HG11) of chain 0 residue 1 (VAL)> VAL
<Atom 17 (HG12) of chain 0 residue 1 (VAL)> VAL
<Atom 18 (HG13) of chain 0 residue 1 (VAL)> VAL
<Atom 19 (HG21) of chain 0 residue 1 (VAL)> VAL
<Atom 20 (HG22) of chain 0 residue 1 (VAL)> VAL
<Atom 21 (HG23) of chain 0 residue 1 

<Atom 1724 (O) of chain 1 residue 115 (HOH)> HOH
<Atom 1725 (H1) of chain 1 residue 115 (HOH)> HOH
<Atom 1726 (H2) of chain 1 residue 115 (HOH)> HOH
<Atom 1727 (O) of chain 1 residue 116 (HOH)> HOH
<Atom 1728 (H1) of chain 1 residue 116 (HOH)> HOH
<Atom 1729 (H2) of chain 1 residue 116 (HOH)> HOH
<Atom 1730 (O) of chain 1 residue 117 (HOH)> HOH
<Atom 1731 (H1) of chain 1 residue 117 (HOH)> HOH
<Atom 1732 (H2) of chain 1 residue 117 (HOH)> HOH
<Atom 1733 (O) of chain 1 residue 118 (HOH)> HOH
<Atom 1734 (H1) of chain 1 residue 118 (HOH)> HOH
<Atom 1735 (H2) of chain 1 residue 118 (HOH)> HOH
<Atom 1736 (O) of chain 1 residue 119 (HOH)> HOH
<Atom 1737 (H1) of chain 1 residue 119 (HOH)> HOH
<Atom 1738 (H2) of chain 1 residue 119 (HOH)> HOH
<Atom 1739 (O) of chain 1 residue 120 (HOH)> HOH
<Atom 1740 (H1) of chain 1 residue 120 (HOH)> HOH
<Atom 1741 (H2) of chain 1 residue 120 (HOH)> HOH
<Atom 1742 (O) of chain 1 residue 121 (HOH)> HOH
<Atom 1743 (H1) of chain 1 residue 121 (HOH)> HOH
<Atom 1

<Atom 3724 (H2) of chain 1 residue 781 (HOH)> HOH
<Atom 3725 (O) of chain 1 residue 782 (HOH)> HOH
<Atom 3726 (H1) of chain 1 residue 782 (HOH)> HOH
<Atom 3727 (H2) of chain 1 residue 782 (HOH)> HOH
<Atom 3728 (O) of chain 1 residue 783 (HOH)> HOH
<Atom 3729 (H1) of chain 1 residue 783 (HOH)> HOH
<Atom 3730 (H2) of chain 1 residue 783 (HOH)> HOH
<Atom 3731 (O) of chain 1 residue 784 (HOH)> HOH
<Atom 3732 (H1) of chain 1 residue 784 (HOH)> HOH
<Atom 3733 (H2) of chain 1 residue 784 (HOH)> HOH
<Atom 3734 (O) of chain 1 residue 785 (HOH)> HOH
<Atom 3735 (H1) of chain 1 residue 785 (HOH)> HOH
<Atom 3736 (H2) of chain 1 residue 785 (HOH)> HOH
<Atom 3737 (O) of chain 1 residue 786 (HOH)> HOH
<Atom 3738 (H1) of chain 1 residue 786 (HOH)> HOH
<Atom 3739 (H2) of chain 1 residue 786 (HOH)> HOH
<Atom 3740 (O) of chain 1 residue 787 (HOH)> HOH
<Atom 3741 (H1) of chain 1 residue 787 (HOH)> HOH
<Atom 3742 (H2) of chain 1 residue 787 (HOH)> HOH
<Atom 3743 (O) of chain 1 residue 788 (HOH)> HOH
<Atom 3

<Atom 5724 (H1) of chain 1 residue 1448 (HOH)> HOH
<Atom 5725 (H2) of chain 1 residue 1448 (HOH)> HOH
<Atom 5726 (O) of chain 1 residue 1449 (HOH)> HOH
<Atom 5727 (H1) of chain 1 residue 1449 (HOH)> HOH
<Atom 5728 (H2) of chain 1 residue 1449 (HOH)> HOH
<Atom 5729 (O) of chain 1 residue 1450 (HOH)> HOH
<Atom 5730 (H1) of chain 1 residue 1450 (HOH)> HOH
<Atom 5731 (H2) of chain 1 residue 1450 (HOH)> HOH
<Atom 5732 (O) of chain 1 residue 1451 (HOH)> HOH
<Atom 5733 (H1) of chain 1 residue 1451 (HOH)> HOH
<Atom 5734 (H2) of chain 1 residue 1451 (HOH)> HOH
<Atom 5735 (O) of chain 1 residue 1452 (HOH)> HOH
<Atom 5736 (H1) of chain 1 residue 1452 (HOH)> HOH
<Atom 5737 (H2) of chain 1 residue 1452 (HOH)> HOH
<Atom 5738 (O) of chain 1 residue 1453 (HOH)> HOH
<Atom 5739 (H1) of chain 1 residue 1453 (HOH)> HOH
<Atom 5740 (H2) of chain 1 residue 1453 (HOH)> HOH
<Atom 5741 (O) of chain 1 residue 1454 (HOH)> HOH
<Atom 5742 (H1) of chain 1 residue 1454 (HOH)> HOH
<Atom 5743 (H2) of chain 1 residue 14

<Atom 7973 (O) of chain 1 residue 2198 (HOH)> HOH
<Atom 7974 (H1) of chain 1 residue 2198 (HOH)> HOH
<Atom 7975 (H2) of chain 1 residue 2198 (HOH)> HOH
<Atom 7976 (O) of chain 1 residue 2199 (HOH)> HOH
<Atom 7977 (H1) of chain 1 residue 2199 (HOH)> HOH
<Atom 7978 (H2) of chain 1 residue 2199 (HOH)> HOH
<Atom 7979 (O) of chain 1 residue 2200 (HOH)> HOH
<Atom 7980 (H1) of chain 1 residue 2200 (HOH)> HOH
<Atom 7981 (H2) of chain 1 residue 2200 (HOH)> HOH
<Atom 7982 (O) of chain 1 residue 2201 (HOH)> HOH
<Atom 7983 (H1) of chain 1 residue 2201 (HOH)> HOH
<Atom 7984 (H2) of chain 1 residue 2201 (HOH)> HOH
<Atom 7985 (O) of chain 1 residue 2202 (HOH)> HOH
<Atom 7986 (H1) of chain 1 residue 2202 (HOH)> HOH
<Atom 7987 (H2) of chain 1 residue 2202 (HOH)> HOH
<Atom 7988 (O) of chain 1 residue 2203 (HOH)> HOH
<Atom 7989 (H1) of chain 1 residue 2203 (HOH)> HOH
<Atom 7990 (H2) of chain 1 residue 2203 (HOH)> HOH
<Atom 7991 (O) of chain 1 residue 2204 (HOH)> HOH
<Atom 7992 (H1) of chain 1 residue 220

<Atom 9223 (H2) of chain 1 residue 2614 (HOH)> HOH
<Atom 9224 (O) of chain 1 residue 2615 (HOH)> HOH
<Atom 9225 (H1) of chain 1 residue 2615 (HOH)> HOH
<Atom 9226 (H2) of chain 1 residue 2615 (HOH)> HOH
<Atom 9227 (O) of chain 1 residue 2616 (HOH)> HOH
<Atom 9228 (H1) of chain 1 residue 2616 (HOH)> HOH
<Atom 9229 (H2) of chain 1 residue 2616 (HOH)> HOH
<Atom 9230 (O) of chain 1 residue 2617 (HOH)> HOH
<Atom 9231 (H1) of chain 1 residue 2617 (HOH)> HOH
<Atom 9232 (H2) of chain 1 residue 2617 (HOH)> HOH
<Atom 9233 (O) of chain 1 residue 2618 (HOH)> HOH
<Atom 9234 (H1) of chain 1 residue 2618 (HOH)> HOH
<Atom 9235 (H2) of chain 1 residue 2618 (HOH)> HOH
<Atom 9236 (O) of chain 1 residue 2619 (HOH)> HOH
<Atom 9237 (H1) of chain 1 residue 2619 (HOH)> HOH
<Atom 9238 (H2) of chain 1 residue 2619 (HOH)> HOH
<Atom 9239 (O) of chain 1 residue 2620 (HOH)> HOH
<Atom 9240 (H1) of chain 1 residue 2620 (HOH)> HOH
<Atom 9241 (H2) of chain 1 residue 2620 (HOH)> HOH
<Atom 9242 (O) of chain 1 residue 262

<Atom 10473 (H1) of chain 1 residue 3031 (HOH)> HOH
<Atom 10474 (H2) of chain 1 residue 3031 (HOH)> HOH
<Atom 10475 (O) of chain 1 residue 3032 (HOH)> HOH
<Atom 10476 (H1) of chain 1 residue 3032 (HOH)> HOH
<Atom 10477 (H2) of chain 1 residue 3032 (HOH)> HOH
<Atom 10478 (O) of chain 1 residue 3033 (HOH)> HOH
<Atom 10479 (H1) of chain 1 residue 3033 (HOH)> HOH
<Atom 10480 (H2) of chain 1 residue 3033 (HOH)> HOH
<Atom 10481 (O) of chain 1 residue 3034 (HOH)> HOH
<Atom 10482 (H1) of chain 1 residue 3034 (HOH)> HOH
<Atom 10483 (H2) of chain 1 residue 3034 (HOH)> HOH
<Atom 10484 (O) of chain 1 residue 3035 (HOH)> HOH
<Atom 10485 (H1) of chain 1 residue 3035 (HOH)> HOH
<Atom 10486 (H2) of chain 1 residue 3035 (HOH)> HOH
<Atom 10487 (O) of chain 1 residue 3036 (HOH)> HOH
<Atom 10488 (H1) of chain 1 residue 3036 (HOH)> HOH
<Atom 10489 (H2) of chain 1 residue 3036 (HOH)> HOH
<Atom 10490 (O) of chain 1 residue 3037 (HOH)> HOH
<Atom 10491 (H1) of chain 1 residue 3037 (HOH)> HOH
<Atom 10492 (H2) o

<Atom 11723 (O) of chain 1 residue 3448 (HOH)> HOH
<Atom 11724 (H1) of chain 1 residue 3448 (HOH)> HOH
<Atom 11725 (H2) of chain 1 residue 3448 (HOH)> HOH
<Atom 11726 (O) of chain 1 residue 3449 (HOH)> HOH
<Atom 11727 (H1) of chain 1 residue 3449 (HOH)> HOH
<Atom 11728 (H2) of chain 1 residue 3449 (HOH)> HOH
<Atom 11729 (O) of chain 1 residue 3450 (HOH)> HOH
<Atom 11730 (H1) of chain 1 residue 3450 (HOH)> HOH
<Atom 11731 (H2) of chain 1 residue 3450 (HOH)> HOH
<Atom 11732 (O) of chain 1 residue 3451 (HOH)> HOH
<Atom 11733 (H1) of chain 1 residue 3451 (HOH)> HOH
<Atom 11734 (H2) of chain 1 residue 3451 (HOH)> HOH
<Atom 11735 (O) of chain 1 residue 3452 (HOH)> HOH
<Atom 11736 (H1) of chain 1 residue 3452 (HOH)> HOH
<Atom 11737 (H2) of chain 1 residue 3452 (HOH)> HOH
<Atom 11738 (O) of chain 1 residue 3453 (HOH)> HOH
<Atom 11739 (H1) of chain 1 residue 3453 (HOH)> HOH
<Atom 11740 (H2) of chain 1 residue 3453 (HOH)> HOH
<Atom 11741 (O) of chain 1 residue 3454 (HOH)> HOH
<Atom 11742 (H1) of

<Atom 13222 (H2) of chain 1 residue 3947 (HOH)> HOH
<Atom 13223 (O) of chain 1 residue 3948 (HOH)> HOH
<Atom 13224 (H1) of chain 1 residue 3948 (HOH)> HOH
<Atom 13225 (H2) of chain 1 residue 3948 (HOH)> HOH
<Atom 13226 (O) of chain 1 residue 3949 (HOH)> HOH
<Atom 13227 (H1) of chain 1 residue 3949 (HOH)> HOH
<Atom 13228 (H2) of chain 1 residue 3949 (HOH)> HOH
<Atom 13229 (O) of chain 1 residue 3950 (HOH)> HOH
<Atom 13230 (H1) of chain 1 residue 3950 (HOH)> HOH
<Atom 13231 (H2) of chain 1 residue 3950 (HOH)> HOH
<Atom 13232 (O) of chain 1 residue 3951 (HOH)> HOH
<Atom 13233 (H1) of chain 1 residue 3951 (HOH)> HOH
<Atom 13234 (H2) of chain 1 residue 3951 (HOH)> HOH
<Atom 13235 (O) of chain 1 residue 3952 (HOH)> HOH
<Atom 13236 (H1) of chain 1 residue 3952 (HOH)> HOH
<Atom 13237 (H2) of chain 1 residue 3952 (HOH)> HOH
<Atom 13238 (O) of chain 1 residue 3953 (HOH)> HOH
<Atom 13239 (H1) of chain 1 residue 3953 (HOH)> HOH
<Atom 13240 (H2) of chain 1 residue 3953 (HOH)> HOH
<Atom 13241 (O) of

<Atom 15722 (O) of chain 1 residue 4781 (HOH)> HOH
<Atom 15723 (H1) of chain 1 residue 4781 (HOH)> HOH
<Atom 15724 (H2) of chain 1 residue 4781 (HOH)> HOH
<Atom 15725 (O) of chain 1 residue 4782 (HOH)> HOH
<Atom 15726 (H1) of chain 1 residue 4782 (HOH)> HOH
<Atom 15727 (H2) of chain 1 residue 4782 (HOH)> HOH
<Atom 15728 (O) of chain 1 residue 4783 (HOH)> HOH
<Atom 15729 (H1) of chain 1 residue 4783 (HOH)> HOH
<Atom 15730 (H2) of chain 1 residue 4783 (HOH)> HOH
<Atom 15731 (O) of chain 1 residue 4784 (HOH)> HOH
<Atom 15732 (H1) of chain 1 residue 4784 (HOH)> HOH
<Atom 15733 (H2) of chain 1 residue 4784 (HOH)> HOH
<Atom 15734 (O) of chain 1 residue 4785 (HOH)> HOH
<Atom 15735 (H1) of chain 1 residue 4785 (HOH)> HOH
<Atom 15736 (H2) of chain 1 residue 4785 (HOH)> HOH
<Atom 15737 (O) of chain 1 residue 4786 (HOH)> HOH
<Atom 15738 (H1) of chain 1 residue 4786 (HOH)> HOH
<Atom 15739 (H2) of chain 1 residue 4786 (HOH)> HOH
<Atom 15740 (O) of chain 1 residue 4787 (HOH)> HOH
<Atom 15741 (H1) of

## Test on D35A

In [7]:
# Read in htf
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/16/12"
htf = pickle.load(open(os.path.join(out_dir, f"12_apo_0.pickle"), "rb" ))

In [8]:
old_topology = htf._topology_proposal.old_topology
new_topology = htf._topology_proposal.new_topology
resid = '35'

In [9]:
# Determine charge of old amino acid
old_res, old_res_name = get_name(old_topology, resid)
old_res_charge = get_charge(old_res_name)

In [10]:
# Determine charge of new amino acid
new_res, new_res_name = get_name(new_topology, resid)
new_res_charge = get_charge(new_res_name)

In [11]:
# Get number and type of ions that need to be turned off
num_ions = get_num_ions(old_res_charge, new_res_charge)

In [12]:
num_ions

1

In [56]:
# Create trajectory
traj = md.Trajectory(np.array(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(new_topology))

water_atoms = list(traj.topology.select(f"water"))

query_atoms = traj.top.select('protein')

# Get water indices within radius of mutated residue
neighboring_atoms = md.compute_neighbors(traj, 0.8, query_atoms, haystack_indices=water_atoms)[0]

# Get water indices outside of radius of mutated residue
# nonneighboring_atoms = [atom for atom in water_atoms if atom not in neighboring_atoms]
nonneighboring_residues = set([atom.residue.index for atom in traj.topology.atoms if atom.index not in neighboring_atoms])
assert len(nonneighboring_residues) > 0, f"there are no available nonneighboring waters"

choice_residues = np.random.choice(list(nonneighboring_residues), size=1, replace=False)
choice_indices = np.array([[atom.index for atom in traj.topology.residue(res).atoms] for res in choice_residues])


<ipython-input-56-76d66e6864cc>:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  neighboring_atoms = md.compute_neighbors(traj, 0.8, query_atoms, haystack_indices=water_atoms)[0]


In [64]:
charge_diff = -1
choice_residues = np.random.choice(list(nonneighboring_residues), size=abs(charge_diff), replace=False)


In [65]:
choice_residues

array([1743])

In [58]:
for atom in traj.topology.atoms:
    if atom.residue.index == 4359:
        print(atom)

HOH4374-O
HOH4374-H1
HOH4374-H2


In [59]:
choice_indices

array([[8189, 8190, 8191]])

In [60]:
np.ndarray.flatten(choice_indices)

array([8189, 8190, 8191])

In [34]:
choice_idx

array([3449])

In [36]:
for atom in htf.hybrid_topology.atoms:
    if atom.residue.index == 3449:
        print(atom)

HOH3461-O
HOH3461-H1
HOH3461-H2


In [29]:
nonneighboring_indices

[1442,
 1443,
 1444,
 1445,
 1446,
 1447,
 1448,
 1449,
 1450,
 1451,
 1452,
 1453,
 1454,
 1455,
 1456,
 1457,
 1458,
 1459,
 1461,
 1463,
 1464,
 1465,
 1466,
 1467,
 1468,
 1472,
 1473,
 1474,
 1475,
 1476,
 1477,
 1481,
 1482,
 1483,
 1484,
 1485,
 1486,
 1487,
 1488,
 1489,
 1490,
 1491,
 1492,
 1493,
 1494,
 1495,
 1496,
 1497,
 1498,
 1499,
 1500,
 1501,
 1502,
 1503,
 1504,
 1511,
 1512,
 1513,
 1514,
 1515,
 1516,
 1520,
 1521,
 1522,
 1523,
 1524,
 1525,
 1526,
 1527,
 1528,
 1529,
 1530,
 1531,
 1532,
 1533,
 1534,
 1538,
 1539,
 1540,
 1541,
 1542,
 1543,
 1547,
 1548,
 1549,
 1553,
 1554,
 1555,
 1556,
 1557,
 1558,
 1562,
 1563,
 1564,
 1565,
 1566,
 1567,
 1568,
 1569,
 1570,
 1574,
 1575,
 1576,
 1577,
 1578,
 1579,
 1586,
 1587,
 1588,
 1592,
 1593,
 1594,
 1595,
 1596,
 1597,
 1598,
 1599,
 1600,
 1601,
 1602,
 1603,
 1604,
 1605,
 1606,
 1607,
 1608,
 1609,
 1613,
 1614,
 1615,
 1616,
 1617,
 1618,
 1619,
 1620,
 1621,
 1622,
 1623,
 1624,
 1625,
 1626,
 1627,
 1631,

In [61]:
residues = [atom.residue.index for atom in traj.topology.atoms if atom.index in atoms]
residues = set(residues)
choice_idx = np.random.choice(nonneighboring_indices, size=abs(charge_diff), replace=False)


NameError: name 'charge_diff' is not defined

In [ ]:
residues

In [33]:
# Get indices of atoms in the new system that need to be turn off
get_indices(new_res, np.array(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(new_topology), num_ions)

[14861]

In [34]:
for atom in new_topology.atoms():
    print(atom, atom.residue.name)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)> ACE
<Atom 1 (C) of chain 0 residue 0 (ACE)> ACE
<Atom 2 (O) of chain 0 residue 0 (ACE)> ACE
<Atom 3 (H1) of chain 0 residue 0 (ACE)> ACE
<Atom 4 (H2) of chain 0 residue 0 (ACE)> ACE
<Atom 5 (H3) of chain 0 residue 0 (ACE)> ACE
<Atom 6 (N) of chain 0 residue 1 (LYS)> LYS
<Atom 7 (CA) of chain 0 residue 1 (LYS)> LYS
<Atom 8 (C) of chain 0 residue 1 (LYS)> LYS
<Atom 9 (O) of chain 0 residue 1 (LYS)> LYS
<Atom 10 (CB) of chain 0 residue 1 (LYS)> LYS
<Atom 11 (CG) of chain 0 residue 1 (LYS)> LYS
<Atom 12 (CD) of chain 0 residue 1 (LYS)> LYS
<Atom 13 (CE) of chain 0 residue 1 (LYS)> LYS
<Atom 14 (NZ) of chain 0 residue 1 (LYS)> LYS
<Atom 15 (H2) of chain 0 residue 1 (LYS)> LYS
<Atom 16 (HA) of chain 0 residue 1 (LYS)> LYS
<Atom 17 (HB3) of chain 0 residue 1 (LYS)> LYS
<Atom 18 (HB2) of chain 0 residue 1 (LYS)> LYS
<Atom 19 (HG2) of chain 0 residue 1 (LYS)> LYS
<Atom 20 (HG3) of chain 0 residue 1 (LYS)> LYS
<Atom 21 (HD2) of chain 0 residue 1 (LYS)> L

<Atom 1591 (H2) of chain 2 residue 141 (HOH)> HOH
<Atom 1592 (O) of chain 2 residue 142 (HOH)> HOH
<Atom 1593 (H1) of chain 2 residue 142 (HOH)> HOH
<Atom 1594 (H2) of chain 2 residue 142 (HOH)> HOH
<Atom 1595 (O) of chain 2 residue 143 (HOH)> HOH
<Atom 1596 (H1) of chain 2 residue 143 (HOH)> HOH
<Atom 1597 (H2) of chain 2 residue 143 (HOH)> HOH
<Atom 1598 (O) of chain 2 residue 144 (HOH)> HOH
<Atom 1599 (H1) of chain 2 residue 144 (HOH)> HOH
<Atom 1600 (H2) of chain 2 residue 144 (HOH)> HOH
<Atom 1601 (O) of chain 2 residue 145 (HOH)> HOH
<Atom 1602 (H1) of chain 2 residue 145 (HOH)> HOH
<Atom 1603 (H2) of chain 2 residue 145 (HOH)> HOH
<Atom 1604 (O) of chain 2 residue 146 (HOH)> HOH
<Atom 1605 (H1) of chain 2 residue 146 (HOH)> HOH
<Atom 1606 (H2) of chain 2 residue 146 (HOH)> HOH
<Atom 1607 (O) of chain 2 residue 147 (HOH)> HOH
<Atom 1608 (H1) of chain 2 residue 147 (HOH)> HOH
<Atom 1609 (H2) of chain 2 residue 147 (HOH)> HOH
<Atom 1610 (O) of chain 2 residue 148 (HOH)> HOH
<Atom 1

<Atom 3591 (H1) of chain 2 residue 808 (HOH)> HOH
<Atom 3592 (H2) of chain 2 residue 808 (HOH)> HOH
<Atom 3593 (O) of chain 2 residue 809 (HOH)> HOH
<Atom 3594 (H1) of chain 2 residue 809 (HOH)> HOH
<Atom 3595 (H2) of chain 2 residue 809 (HOH)> HOH
<Atom 3596 (O) of chain 2 residue 810 (HOH)> HOH
<Atom 3597 (H1) of chain 2 residue 810 (HOH)> HOH
<Atom 3598 (H2) of chain 2 residue 810 (HOH)> HOH
<Atom 3599 (O) of chain 2 residue 811 (HOH)> HOH
<Atom 3600 (H1) of chain 2 residue 811 (HOH)> HOH
<Atom 3601 (H2) of chain 2 residue 811 (HOH)> HOH
<Atom 3602 (O) of chain 2 residue 812 (HOH)> HOH
<Atom 3603 (H1) of chain 2 residue 812 (HOH)> HOH
<Atom 3604 (H2) of chain 2 residue 812 (HOH)> HOH
<Atom 3605 (O) of chain 2 residue 813 (HOH)> HOH
<Atom 3606 (H1) of chain 2 residue 813 (HOH)> HOH
<Atom 3607 (H2) of chain 2 residue 813 (HOH)> HOH
<Atom 3608 (O) of chain 2 residue 814 (HOH)> HOH
<Atom 3609 (H1) of chain 2 residue 814 (HOH)> HOH
<Atom 3610 (H2) of chain 2 residue 814 (HOH)> HOH
<Atom 

<Atom 5591 (O) of chain 2 residue 1475 (HOH)> HOH
<Atom 5592 (H1) of chain 2 residue 1475 (HOH)> HOH
<Atom 5593 (H2) of chain 2 residue 1475 (HOH)> HOH
<Atom 5594 (O) of chain 2 residue 1476 (HOH)> HOH
<Atom 5595 (H1) of chain 2 residue 1476 (HOH)> HOH
<Atom 5596 (H2) of chain 2 residue 1476 (HOH)> HOH
<Atom 5597 (O) of chain 2 residue 1477 (HOH)> HOH
<Atom 5598 (H1) of chain 2 residue 1477 (HOH)> HOH
<Atom 5599 (H2) of chain 2 residue 1477 (HOH)> HOH
<Atom 5600 (O) of chain 2 residue 1478 (HOH)> HOH
<Atom 5601 (H1) of chain 2 residue 1478 (HOH)> HOH
<Atom 5602 (H2) of chain 2 residue 1478 (HOH)> HOH
<Atom 5603 (O) of chain 2 residue 1479 (HOH)> HOH
<Atom 5604 (H1) of chain 2 residue 1479 (HOH)> HOH
<Atom 5605 (H2) of chain 2 residue 1479 (HOH)> HOH
<Atom 5606 (O) of chain 2 residue 1480 (HOH)> HOH
<Atom 5607 (H1) of chain 2 residue 1480 (HOH)> HOH
<Atom 5608 (H2) of chain 2 residue 1480 (HOH)> HOH
<Atom 5609 (O) of chain 2 residue 1481 (HOH)> HOH
<Atom 5610 (H1) of chain 2 residue 148

<Atom 7840 (H2) of chain 2 residue 2224 (HOH)> HOH
<Atom 7841 (O) of chain 2 residue 2225 (HOH)> HOH
<Atom 7842 (H1) of chain 2 residue 2225 (HOH)> HOH
<Atom 7843 (H2) of chain 2 residue 2225 (HOH)> HOH
<Atom 7844 (O) of chain 2 residue 2226 (HOH)> HOH
<Atom 7845 (H1) of chain 2 residue 2226 (HOH)> HOH
<Atom 7846 (H2) of chain 2 residue 2226 (HOH)> HOH
<Atom 7847 (O) of chain 2 residue 2227 (HOH)> HOH
<Atom 7848 (H1) of chain 2 residue 2227 (HOH)> HOH
<Atom 7849 (H2) of chain 2 residue 2227 (HOH)> HOH
<Atom 7850 (O) of chain 2 residue 2228 (HOH)> HOH
<Atom 7851 (H1) of chain 2 residue 2228 (HOH)> HOH
<Atom 7852 (H2) of chain 2 residue 2228 (HOH)> HOH
<Atom 7853 (O) of chain 2 residue 2229 (HOH)> HOH
<Atom 7854 (H1) of chain 2 residue 2229 (HOH)> HOH
<Atom 7855 (H2) of chain 2 residue 2229 (HOH)> HOH
<Atom 7856 (O) of chain 2 residue 2230 (HOH)> HOH
<Atom 7857 (H1) of chain 2 residue 2230 (HOH)> HOH
<Atom 7858 (H2) of chain 2 residue 2230 (HOH)> HOH
<Atom 7859 (O) of chain 2 residue 223

<Atom 10090 (H2) of chain 2 residue 2974 (HOH)> HOH
<Atom 10091 (O) of chain 2 residue 2975 (HOH)> HOH
<Atom 10092 (H1) of chain 2 residue 2975 (HOH)> HOH
<Atom 10093 (H2) of chain 2 residue 2975 (HOH)> HOH
<Atom 10094 (O) of chain 2 residue 2976 (HOH)> HOH
<Atom 10095 (H1) of chain 2 residue 2976 (HOH)> HOH
<Atom 10096 (H2) of chain 2 residue 2976 (HOH)> HOH
<Atom 10097 (O) of chain 2 residue 2977 (HOH)> HOH
<Atom 10098 (H1) of chain 2 residue 2977 (HOH)> HOH
<Atom 10099 (H2) of chain 2 residue 2977 (HOH)> HOH
<Atom 10100 (O) of chain 2 residue 2978 (HOH)> HOH
<Atom 10101 (H1) of chain 2 residue 2978 (HOH)> HOH
<Atom 10102 (H2) of chain 2 residue 2978 (HOH)> HOH
<Atom 10103 (O) of chain 2 residue 2979 (HOH)> HOH
<Atom 10104 (H1) of chain 2 residue 2979 (HOH)> HOH
<Atom 10105 (H2) of chain 2 residue 2979 (HOH)> HOH
<Atom 10106 (O) of chain 2 residue 2980 (HOH)> HOH
<Atom 10107 (H1) of chain 2 residue 2980 (HOH)> HOH
<Atom 10108 (H2) of chain 2 residue 2980 (HOH)> HOH
<Atom 10109 (O) of

<Atom 12340 (H2) of chain 2 residue 3724 (HOH)> HOH
<Atom 12341 (O) of chain 2 residue 3725 (HOH)> HOH
<Atom 12342 (H1) of chain 2 residue 3725 (HOH)> HOH
<Atom 12343 (H2) of chain 2 residue 3725 (HOH)> HOH
<Atom 12344 (O) of chain 2 residue 3726 (HOH)> HOH
<Atom 12345 (H1) of chain 2 residue 3726 (HOH)> HOH
<Atom 12346 (H2) of chain 2 residue 3726 (HOH)> HOH
<Atom 12347 (O) of chain 2 residue 3727 (HOH)> HOH
<Atom 12348 (H1) of chain 2 residue 3727 (HOH)> HOH
<Atom 12349 (H2) of chain 2 residue 3727 (HOH)> HOH
<Atom 12350 (O) of chain 2 residue 3728 (HOH)> HOH
<Atom 12351 (H1) of chain 2 residue 3728 (HOH)> HOH
<Atom 12352 (H2) of chain 2 residue 3728 (HOH)> HOH
<Atom 12353 (O) of chain 2 residue 3729 (HOH)> HOH
<Atom 12354 (H1) of chain 2 residue 3729 (HOH)> HOH
<Atom 12355 (H2) of chain 2 residue 3729 (HOH)> HOH
<Atom 12356 (O) of chain 2 residue 3730 (HOH)> HOH
<Atom 12357 (H1) of chain 2 residue 3730 (HOH)> HOH
<Atom 12358 (H2) of chain 2 residue 3730 (HOH)> HOH
<Atom 12359 (O) of

<Atom 14590 (H2) of chain 2 residue 4474 (HOH)> HOH
<Atom 14591 (O) of chain 2 residue 4475 (HOH)> HOH
<Atom 14592 (H1) of chain 2 residue 4475 (HOH)> HOH
<Atom 14593 (H2) of chain 2 residue 4475 (HOH)> HOH
<Atom 14594 (O) of chain 2 residue 4476 (HOH)> HOH
<Atom 14595 (H1) of chain 2 residue 4476 (HOH)> HOH
<Atom 14596 (H2) of chain 2 residue 4476 (HOH)> HOH
<Atom 14597 (O) of chain 2 residue 4477 (HOH)> HOH
<Atom 14598 (H1) of chain 2 residue 4477 (HOH)> HOH
<Atom 14599 (H2) of chain 2 residue 4477 (HOH)> HOH
<Atom 14600 (O) of chain 2 residue 4478 (HOH)> HOH
<Atom 14601 (H1) of chain 2 residue 4478 (HOH)> HOH
<Atom 14602 (H2) of chain 2 residue 4478 (HOH)> HOH
<Atom 14603 (O) of chain 2 residue 4479 (HOH)> HOH
<Atom 14604 (H1) of chain 2 residue 4479 (HOH)> HOH
<Atom 14605 (H2) of chain 2 residue 4479 (HOH)> HOH
<Atom 14606 (O) of chain 2 residue 4480 (HOH)> HOH
<Atom 14607 (H1) of chain 2 residue 4480 (HOH)> HOH
<Atom 14608 (H2) of chain 2 residue 4480 (HOH)> HOH
<Atom 14609 (O) of

##  Make the above into one function

In [42]:
def get_counterion_indices(htf, resid, positive_ion_name='NA', negative_ion_name='CL', radius=0.5):
    
    # Get old and new topologies
    old_topology = htf._topology_proposal.old_topology
    new_topology = htf._topology_proposal.new_topology
    
    # Get the residue objects and residue names of the mutated residue, given the old and new topologies and the resid
    for res in old_topology.residues():
        if res.id == resid:
            old_res = res
            old_res_name = res.name
    if not old_res:
        raise Exception(f"Could not find resid {resid} in old topology!")
    
    for res in new_topology.residues():
        if res.id == resid:
            new_res = res
            new_res_name = res.name
    if not new_res:
        raise Exception(f"Could not find resid {resid} in new topology!")

    # Get the charge of an residue, given its name
    charges = []
    for res_name in [old_res_name, new_res_name]:
        if res_name in positive:
            charges.append(1)
        elif res_name in negative: 
            charges.append(-1)
        else:
            charges.append(0)

    # Determine how many ions needs to be turned off
    charge_difference = charges[0] - charges[1]
    if charge_difference == 0:
        num_ions = 0
    else:
        num_ions = charge_difference*-1

    # Retrieve atom indices of mutated residue for querying neighboring atoms
    query_indices = [atom.index for atom in new_res.atoms()]
    
    # Create trajectory
    traj = md.Trajectory(np.array(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(new_topology))
    
    # Get ion atom indices
    if num_ions < 0:
        ion_name = negative_ion_name
    elif num_ions > 0:
        ion_name = positive_ion_name
    atoms = list(traj.topology.select(f"resname {ion_name}"))
    
    # Get ion indices within radius of mutated residue
    neighboring_ions = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=atoms)[0])
    
    # Get ion indices outside of radius of mutated residue
    nonneighboring_ions = list(set(atoms) - set(neighboring_ions))
    
    return nonneighboring_ions[:abs(num_ions)] # Return first N ion indices, where N = abs(num_ions)

## Test on A27K

In [38]:
# Read in htf
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/12/57"
htf = pickle.load(open(os.path.join(out_dir, f"57_apo_0.pickle"), "rb" ))

In [43]:
get_counterion_indices(htf, '27')

[18096]